In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
# create spark configuration
conf = SparkConf()
# use two kernels to speed up
conf.setMaster('local[2]')
conf.setAppName("TwitterStreamApp")
# create spark context with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
# create the Streaming Context from the above spark context with interval size 2 seconds
ssc = StreamingContext(sc, 2)
# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_TwitterApp")

In [2]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        print("start")
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        print("sql_context")
        sql_context.show()
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        print("row_rdd")
        row_rdd.show()
        # create a DF from the Row RDD
        hashtags_df = sql_context.createDataFrame(row_rdd)
        print("hashtags_df")
        hashtags_df.show()
        # Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        print("hashtages")
        print(hashtages.show())
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        print("hashtag_count_df")
        hashtag_counts_df.show()
    except:
        e = sys.exc_info()[0]
        print(e)
#         finally:
#             try:
#                 hashtag_counts_df.show()
#             except:
#                 pass

def send_df_to_dashboard(df):
    # extract the hashtags from dataframe and convert them into array
    top_tags = [str(t.hashtag) for t in df.select("hashtag").collect()]
    # extract the counts from dataframe and convert them into array
    tags_count = [p.hashtag_count for p in df.select("hashtag_count").collect()]
    # initialize and send the data through REST API
    url = 'http://localhost:5001/updateData'
    request_data = {'label': str(top_tags), 'data': str(tags_count)}
    response = requests.post(url, data=request_data)

In [3]:
# read data from port 9009
dataStream = ssc.socketTextStream("localhost",10005)

In [4]:
dataStream.pprint()

In [5]:
# split each tweet into words
words = dataStream.flatMap(lambda line: line.split(" "))
words.pprint()
# filter the words to get only hashtags, then map each hashtag to be a pair of (hashtag,1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))
# adding the count of each hashtag to its last count
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)
# start the streaming computation
ssc.start()
# wait for the streaming to finish
ssc.awaitTermination()

-------------------------------------------
Time: 2018-10-20 00:32:30
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 00:32:30
-------------------------------------------

----------- 2018-10-20 00:32:30 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time: 2018-10-20 00:32:32
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 00:32:32
-------------------------------------------

----------- 2018-10-20 00:32:32 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time: 2018-10-20 00:32:34
-------------------------------------------
Bring #SecondAct to #NewYork 🗽!!! Want to be the first to see #SecondAct? Follow these instructions and vote below… https://t.co/NZkvuCNlzp
@DawnNeufeld #roevswade the topic .. gicha mind right #chickrnfingers .:) #rp aka #mj https://t.co/zes3yEpojO
Po

-------------------------------------------
Time: 2018-10-20 00:32:48
-------------------------------------------
@GeraldoRivera @POTUS Our @POTUS is a compromised disgrace. America used to be the moral leader, with #TraitorTrump… https://t.co/R5OHG9nv1e
@ericjvmes_ Lol well it just started
Great start in Illinois. NCCC beat #7 ranked Grand Rapids 3-0! #SHOtime
Isn’t she Lucky? At the happiest place on earth. Bingo with lindalovesbingonyc!!!! With @simpsonlinda &amp; timmurphyny… https://t.co/ivCyd1HkwZ
And I understand why they were so aggressive when they would say it😂😂 shit stinks 😂😂
@tosinakande__ It is not time yet.
Seriously TIRED of people thinking it’s okay to “walk away” whether serious or not. I’ve gone through it enough. I’m over it
Stephanie’s is bomb yo 😭❤️
@ballroombeauty7 @stmoriz Will do! x
"ew you really eat cheesecake?" https://t.co/cMG8aVcHWO
...

-------------------------------------------
Time: 2018-10-20 00:32:48
-------------------------------------------
@Gerald

-------------------------------------------
Time: 2018-10-20 00:33:04
-------------------------------------------
@WWE. @VinceMcMahon you cannot go to Saudi
They are barbaric. If you do blood will be on your hands
 #khashoggi
Hello everyone. This is Nugget. We are fostering her until she goes into heat and then hopefully getting a puppy fr… https://t.co/XGBCpH1lmp
@HoarseWisperer Take a deep breathe and just think about the fact Donald Trump will not be president anymore someda… https://t.co/hbQtTVflv1
@every_daydad You can still do it
pretty teeth w a nice demeanor
This is an intense IPA, won’t need more than one of these. - Drinking a Rodeo Clown Double IPA by @karbachbrewing a… https://t.co/OSnXAv3obT
@kaelinlorrraine Yeah mhm anyway 
Text me ya address
...

-------------------------------------------
Time: 2018-10-20 00:33:04
-------------------------------------------
@WWE.
@VinceMcMahon
you
cannot
go
to
Saudi
They
are
barbaric.
...

----------- 2018-10-20 00:33:04 -----------
sta

KeyboardInterrupt: 

-------------------------------------------
Time: 2018-10-20 00:33:10
-------------------------------------------
AYE SOMEBODY MATCH!!!!
IM FUCKING NERVOUS BRO
@stoolpresidente ever see a man’s soul leave his body? https://t.co/BLGdjwCz43
The music 😂😂
Guys this is a tremendous hockey game @UMassHockey
My team, taking care of the associates in national cupcake day. @LaurenSmith0929 @senecal_roger @Bill1501L… https://t.co/rSs83h5Qn6
CBUS ➡️ KCMO for date weekend with a certain hot blonde and a killer #NFL matchup on Sunday. 

@Joe_MainMixon,… https://t.co/EDbzTC3amp
::sigh:: I hate Joe Buck.
...

-------------------------------------------
Time: 2018-10-20 00:33:10
-------------------------------------------
AYE
SOMEBODY
MATCH!!!!
IM
FUCKING
NERVOUS
BRO
@stoolpresidente
ever
see
...

----------- 2018-10-20 00:33:10 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time: 2018-10-20 00:33:12
-------------------------------------------
At the

...

-------------------------------------------
Time: 2018-10-20 01:01:30
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 01:01:30
-------------------------------------------

----------- 2018-10-20 01:01:30 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time: 2018-10-20 01:01:32
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 01:01:32
-------------------------------------------

----------- 2018-10-20 01:01:32 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time: 2018-10-20 01:01:34
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 01:01:34
-------------------------------------------

----------- 2018-10-20 01:01:34 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time:

-------------------------------------------
Time: 2018-10-20 01:02:22
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 01:02:22
-------------------------------------------

----------- 2018-10-20 01:02:22 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time: 2018-10-20 01:02:24
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 01:02:24
-------------------------------------------

----------- 2018-10-20 01:02:24 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time: 2018-10-20 01:02:26
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 01:02:26
-------------------------------------------

----------- 2018-10-20 01:02:26 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time:

-------------------------------------------
Time: 2018-10-20 01:03:14
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 01:03:14
-------------------------------------------

----------- 2018-10-20 01:03:14 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time: 2018-10-20 01:03:16
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 01:03:16
-------------------------------------------

----------- 2018-10-20 01:03:16 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time: 2018-10-20 01:03:18
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 01:03:18
-------------------------------------------

----------- 2018-10-20 01:03:18 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time:

-------------------------------------------
Time: 2018-10-20 01:04:06
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 01:04:06
-------------------------------------------

----------- 2018-10-20 01:04:06 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time: 2018-10-20 01:04:08
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 01:04:08
-------------------------------------------

----------- 2018-10-20 01:04:08 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time: 2018-10-20 01:04:10
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 01:04:10
-------------------------------------------

----------- 2018-10-20 01:04:10 -----------
start
sql_context
<class 'AttributeError'>
-------------------------------------------
Time:

-------------------------------------------
Time: 2018-10-20 01:04:58
-------------------------------------------

-------------------------------------------
Time: 2018-10-20 01:04:58
-------------------------------------------

----------- 2018-10-20 01:04:58 -----------
start
sql_context
<class 'AttributeError'>
